## Usando arquivo .env para controlar variaveis de ambiente
Para evitar exposição da chave `OPENAI_API_KEY` optei por utilizar arquivo `.env` com a informação da chave.

Para seguir o mesmo método basta criar um arquivo `.env` no mesmo diretório do arquivo `my-first-crew.ipynb`.
A importação da chave será feita através da célula abaixo que faz a instalação de um biblioteca para carregar
os valores do arquivo `.env`.

In [ ]:
%%capture
%pip install python-dotenv
import dotenv
%load_ext dotenv
%dotenv

In [1]:
%%capture
%pip install crewai langchain

In [2]:
from crewai import Agent, Task, Crew


Exception while exporting Span batch.
Traceback (most recent call last):
  File "/Users/iasmim.santos/ai/tdc-crewai-workshop/.venv/lib/python3.12/site-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/iasmim.santos/ai/tdc-crewai-workshop/.venv/lib/python3.12/site-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/Users/iasmim.santos/ai/tdc-crewai-workshop/.venv/lib/python3.12/site-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
OSError: [Errno 9] Bad file descriptor

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/iasmim.santos/ai/tdc-crewai-workshop/.venv/lib/python3.12/site-packages/urllib3/connectionpool.py", line 787, in urlopen
    response = self._make_request(
               ^^^^^^^^^^^^^^^^^^^
  File "/Users/iasmim.santos/ai/t

In [3]:
import os
os.environ['OPENAI_MODEL_NAME'] = "gpt-3.5-turbo"
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")

# Grupo de agentes para revisão de código

#### 1. CodeQualityAgent - Analisa a qualidade do código


In [4]:
code_quality_agent = Agent(
    role="Especialista em Qualidade de Código",
    goal="Analisar o código para identificar problemas de qualidade e sugerir melhorias",
    backstory="""Você é um desenvolvedor sênior com vasta experiência em análise de código.
    Sua expertise está em identificar código duplicado, funções muito longas, 
    complexidade excessiva, e outros problemas que afetam a qualidade e manutenibilidade.
    Você é conhecido por fornecer sugestões práticas e objetivas para melhorar a qualidade do código.
    """,
    allow_delegation=False,
    verbose=True,
)

#### 2. SecurityAgent - Analisa vulnerabilidades de segurança


In [5]:
security_agent = Agent(
    role="Especialista em Segurança",
    goal="Identificar vulnerabilidades de segurança no código e sugerir correções",
    backstory="""Você é um especialista em segurança de aplicações com experiência em 
    identificar vulnerabilidades como injeção, XSS, problemas de autenticação, 
    e outras falhas de segurança comuns.
    Sua análise é minuciosa e você fornece recomendações específicas 
    sobre como corrigir problemas de segurança encontrados.
    """,
    allow_delegation=False,
    verbose=True,
)


#### 3. StyleAgent - Analisa aderência a padrões de código


In [6]:
style_agent = Agent(
    role="Especialista em Padrões de Código",
    goal="Verificar a aderência do código aos padrões de estilo e melhores práticas",
    backstory="""Você é um especialista em estilo de código e convenções de programação.
    Sua atenção a detalhes permite identificar inconsistências na formatação, nomenclatura, 
    e organização do código. Você é apaixonado por código limpo e bem estruturado, seguindo 
    as melhores práticas da indústria e padrões específicos da linguagem.
    """,
    allow_delegation=False,
    verbose=True,
)

In [7]:
quality_review_task = Task(
    description="""Analise o arquivo sample_code.tsx e identifique:
    1. Código duplicado ou redundante
    2. Funções ou métodos que são muito longos ou complexos
    3. Problemas de abstração ou encapsulamento
    4. Oportunidades para melhorar a eficiência do código
    5. Problemas com a estrutura geral do código
    
    Forneça um relatório detalhado com exemplos específicos do código e sugestões
    claras para melhorias, priorizadas por impacto. Para cada sugestão, explique o
    problema e como a solução proposta melhora a manutenibilidade, legibilidade
    ou desempenho do código.
    """,
    expected_output="Um relatório detalhado de problemas de qualidade de código e recomendações específicas de melhorias",
    agent=code_quality_agent,
)

In [8]:
security_review_task = Task(
    description="""Analise o arquivo sample_code.tsx para vulnerabilidades de segurança, incluindo:
    1. Vulnerabilidades a ataques XSS ou injeção
    2. Vazamento de informações sensíveis
    3. Problemas de autenticação ou autorização
    4. Validação inadequada de entrada
    5. Outras vulnerabilidades comuns em aplicações web
    
    Forneça um relatório de segurança que priorize as vulnerabilidades por gravidade,
    explicando cada problema encontrado, seu potencial impacto e recomendações 
    específicas sobre como corrigir cada vulnerabilidade.
    """,
    expected_output="Um relatório de segurança com vulnerabilidades identificadas, classificadas por severidade, e recomendações de correção",
    agent=security_agent,
)

In [9]:
style_review_task = Task(
    description="""Analise o arquivo sample_code.tsx para verificar a aderência a padrões de estilo:
    1. Consistência na nomenclatura de variáveis, funções e componentes
    2. Formatação adequada (indentação, espaçamento, etc.)
    3. Organização de imports e dependências
    4. Comentários adequados e documentação
    5. Aderência a padrões e melhores práticas específicas do React/TypeScript
    
    Forneça um relatório detalhado sobre problemas encontrados, explicando por
    que são considerados desvios de boas práticas e como corrigi-los para melhorar
    a legibilidade e manutenibilidade do código.
    """,
    expected_output="Um relatório sobre problemas de estilo e padronização, com recomendações para alinhar o código às melhores práticas",
    agent=style_agent,
)

In [10]:
review_crew = Crew(
    agents=[code_quality_agent, security_agent, style_agent],
    tasks=[quality_review_task, security_review_task, style_review_task],
    verbose=True,
)

In [11]:
with open('sample_code.tsx', 'r') as file:
    code_content = file.read()

In [12]:
result = review_crew.kickoff(inputs={"code": code_content})


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 5d8caa61-be2e-4505-9987-91e59a086fea                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 221eb611-097e-4948-9e28-7344e7227f3a
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 221eb611-097e-4948-9e28-7344e7227f3a
       Status: Executing Task...
    └── 🤖 Agent: Especialista em Qualidade de Código
            Status: In Progress

# Agent: Especialista em Qualidade de Código
## Task: Analise o arquivo sample_code.tsx e identifique:
    1. Código duplicado ou redundante
    2. Funções ou métodos que são muito longos ou complexos
    3. Problemas de abstração ou encapsulamento
    4. Oportunidades para melhorar a eficiência do código
    5. Problemas com a estrutura geral do código

    Forneça um relatório detalhado com exemplos específicos do código e sugestões
    claras para melhorias, priorizadas por impacto. Para cada sugestão, explique o
    problema e como a solução proposta melhora a manutenibilidade, legibilidade
    ou desempenho do código.
    


🚀 Crew: crew
└── 📋 Task: 221eb611-097e-4948-9e28-7344e7227f3a
       Status: Executing Task...
    └── 🤖 Agent: Especialista em Qualidade de Código
            Status: In Progress
        └── 🧠 Thinking...

...



# Agent: Especialista em Padrões de Código
## Final Answer: 
# Relatório de Análise de Estilo e Boas Práticas - sample_code.tsx

## 1. Inconsistência na nomenclatura:
- Problema: Uso de nomenclaturas inconsistentes para variáveis e métodos. Exemplo:
```javascript
// Inconsistência entre totalPrice e total
const totalPrice = price * quantity;
setTotal(total);
```
Recomendação: Padronizar a nomenclatura utilizando camelCase para variáveis e métodos, mantendo um estilo uniforme em todo o código.

## 2. Formatação inadequada:
- Problema: Espaçamento inconsistente e má indentação no código. Exemplo:
```javascript
// Indentação inconsistente
if(condition){
// código sem espaçamento adequado
}
```
Recomendação: Utilizar um padrão de indentação de espaços (por exemplo, 2 ou 4 espaços) de forma consistente em todo o código para melhorar a legibilidade.

## 3. Organização de imports:
- Problema: Imports desorganizados e não agrupados adequadamente. Exemplo:
```javascript
import React, { useSta

🚀 Crew: crew
├── 📋 Task: 221eb611-097e-4948-9e28-7344e7227f3a
│      Assigned to: Especialista em Qualidade de Código
│      Status: ✅ Completed
│   └── 🤖 Agent: Especialista em Qualidade de Código
│           Status: ✅ Completed
├── 📋 Task: 149b3d08-aca1-499e-a5d6-e4e80dcda2e1
│      Assigned to: Especialista em Segurança
│      Status: ✅ Completed
│   └── 🤖 Agent: Especialista em Segurança
│           Status: ✅ Completed
└── 📋 Task: 52bc8b85-5de4-484a-9b99-72e6562d56a6
       Status: Executing Task...
    └── 🤖 Agent: Especialista em Padrões de Código
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 221eb611-097e-4948-9e28-7344e7227f3a
│      Assigned to: Especialista em Qualidade de Código
│      Status: ✅ Completed
│   └── 🤖 Agent: Especialista em Qualidade de Código
│           Status: ✅ Completed
├── 📋 Task: 149b3d08-aca1-499e-a5d6-e4e80dcda2e1
│      Assigned to: Especialista em Segurança
│      Status: ✅ Completed
│   └── 🤖 Agent: Especialista em Segurança
│           Status: ✅ Completed
└── 📋 Task: 52bc8b85-5de4-484a-9b99-72e6562d56a6
       Assigned to: Especialista em Padrões de Código
       Status: ✅ Completed
    └── 🤖 Agent: Especialista em Padrões de Código
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 52bc8b85-5de4-484a-9b99-72e6562d56a6                                                                     │
│  Agent: Especialista em Padrões de Código                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 5d8caa61-be2e-4505-9987-91e59a086fea                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [13]:
from IPython.display import Markdown

In [14]:
Markdown(result.raw)

# Relatório de Análise de Estilo e Boas Práticas - sample_code.tsx

## 1. Inconsistência na nomenclatura:
- Problema: Uso de nomenclaturas inconsistentes para variáveis e métodos. Exemplo:
```javascript
// Inconsistência entre totalPrice e total
const totalPrice = price * quantity;
setTotal(total);
```
Recomendação: Padronizar a nomenclatura utilizando camelCase para variáveis e métodos, mantendo um estilo uniforme em todo o código.

## 2. Formatação inadequada:
- Problema: Espaçamento inconsistente e má indentação no código. Exemplo:
```javascript
// Indentação inconsistente
if(condition){
// código sem espaçamento adequado
}
```
Recomendação: Utilizar um padrão de indentação de espaços (por exemplo, 2 ou 4 espaços) de forma consistente em todo o código para melhorar a legibilidade.

## 3. Organização de imports:
- Problema: Imports desorganizados e não agrupados adequadamente. Exemplo:
```javascript
import React, { useState, useEffect } from 'react';
import OtherComponent from './OtherComponent';
import { constant1, constant2 } from './constants';
```
Recomendação: Agrupar os imports por categorias (bibliotecas externas, componentes locais, constantes, etc.) e manter uma ordem alfabética para facilitar a leitura e manutenção dos imports.

## 4. Comentários e documentação:
- Problema: Falta de comentários explicativos e documentação sobre o código. Exemplo:
```javascript
// Comentário vago e pouco informativo
const x = 10;
```
Recomendação: Incluir comentários claros e concisos para explicar a lógica complexa, intenções de código e documentar funções ou componentes para facilitar a compreensão do código por outros desenvolvedores.

## 5. Aderência a padrões do React/TypeScript:
- Problema: Não seguimento de padrões específicos do React/TypeScript, como props tipadas de forma inadequada. Exemplo:
```javascript
// Props não tipadas ou tipadas de forma genérica
interface MyComponentProps {
  name: string;
  age: number;
}
```
Recomendação: Utilizar prop-types ou PropTypes para tipar de forma apropriada as props dos componentes React, além de adotar práticas como uso de hooks de forma consistente.

A correção desses problemas contribuirá significativamente para a qualidade, legibilidade e manutenibilidade do código, além de garantir a consistência e aderência aos padrões de estilo e práticas recomendadas. É fundamental revisar e aplicar as recomendações mencionadas gradualmente para aprimorar o código de acordo com as melhores práticas da indústria.